In [ ]:
a=Sys.time()

library(parallel)
source('/nas/cee-water/cjgleason/calval_toolbox/R code/calculate_area_from_imagery.R')

utm_zone = 10
#scale max width (in cases of under/overestimated or centerline offset)
scale_maxwidth = 3
datetime=as.POSIXct('2022-07-29 19:00:00')
##other parameter setting
#Threshold for water mapping
water_index_threshold = 0.2
reach_or_node='node'

#path of input image 
image_list = paste0('/nas/cee-water/cjgleason/calval/willamette_geotiffs/raw/WillametteROI/',
                    list.files('/nas/cee-water/cjgleason/calval/willamette_geotiffs/raw/WillametteROI/'))

#path of input SWORD data (netcdf)
SWORD_path='/nas/cee-water/cjgleason/SWORDv14/Reaches_Nodes/netcdf/na_sword_v11.nc'
#output dir
dir_output = '/nas/cee-water/cjgleason/calval_toolbox/Willamette/Watermask/'

# #selected reach and nodes to be processed
this_river_reach_ids <- c(78220000191,78220000181,78220000171)
this_river_node_ids <- c(78220000190011,78220000180811,78220000180661,78220000180511,
                        78220000180351,78220000180341,78220000180201,
                         78220000180191,78220000170571,78220000170401,78220000170281,78220000170161,78220000170011)

reach=c(78220000191,
        78220000181,78220000181,78220000181,78220000181,78220000181,78220000181,78220000181,
        78220000171,78220000171,78220000171,78220000171,78220000171,
        78220000191,78220000181,78220000171)
        
node=c(78220000190011, 
       78220000180811,78220000180661,78220000180511,78220000180351,78220000180341,78220000180201,78220000180191,
       78220000170571,78220000170401,78220000170281,78220000170161,78220000170011,
       'NA','NA','NA')

reach_or_node=c(rep('node',times=13),rep('reach',times=3))

# reach=c(78220000191)
        
# node=c(78220000190011)

# reach_or_node='node'


df=data.frame(reach=reach,node=node,reach_or_node=reach_or_node)
 input_list=do.call("mapply", c(list, df, SIMPLIFY = FALSE, USE.NAMES=FALSE))
#print(input_list)
 cl=makeCluster(16)
dummy=do.call(rbind,parLapply(cl,input_list,calculate_area_from_imagery,image_list=image_list,
                                     utm_zone=utm_zone,
                                     datetime=datetime,
                                     scale_maxwidth=scale_maxwidth, 
                                     SWORD_path=SWORD_path,
                                     water_index_threshold=water_index_threshold,
                                    dir_output=dir_output  ))
 write.csv(dummy,paste0(dir_output,'CSV/','willamete_epxeriment_area_df.csv'))
stopCluster(cl)

# calculate_area_from_imagery(input_list=input_list,
#                             image_list=image_list,
#                                      utm_zone=utm_zone,
#                                      datetime=datetime,
#                                      scale_maxwidth=scale_maxwidth, 
#                                      SWORD_path=SWORD_path,
#                                      water_index_threshold=water_index_threshold,
#                                     dir_output=dir_output  )
print(Sys.time()-a)#calcluate areas from images----


In [2]:
 write.csv(dummy,paste0(dir_output,'CSV/','willamete_epxeriment_area_df.csv'))